# Marketing A/B Test - Exploratory Data Analysis

**Author**: Analytics Team  
**Date**: November 2025  
**Version**: 1.0

---

## Executive Summary

This notebook performs comprehensive exploratory data analysis (EDA) on the marketing campaign data. The analysis includes data quality assessment, conversion rate analysis, temporal pattern identification, and dose-response relationship investigation.

## Objectives

1. **Data Quality Assessment**: Verify data completeness, check for missing values, duplicates, and data consistency
2. **Conversion Rate Analysis**: Calculate and compare conversion rates between test and control groups
3. **Temporal Pattern Analysis**: Identify optimal timing for ad delivery (day/hour effects)
4. **Dose-Response Analysis**: Investigate the relationship between ad exposure frequency and conversion rates

## Methodology

We use descriptive statistics, correlation analysis, and visualization techniques to explore the data and identify key patterns that inform subsequent statistical testing.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import spearmanr, pearsonr
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries loaded successfully!")


## 1. Data Loading & Initial Inspection

First, we load the dataset and perform initial inspection to understand its structure.


In [ ]:
# Load data
df = pd.read_csv('marketing_AB.csv')

print("="*80)
print("DATASET OVERVIEW")
print("="*80)
print(f"Total Records: {len(df):,}")
print(f"Features: {df.shape[1]}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst 5 Rows:")
df.head()


## 2. Data Quality Assessment

We check for:
- Missing values
- Duplicate records
- Data consistency
- Unique user verification


In [ ]:
# Missing values
print("MISSING VALUES:")
missing = df.isnull().sum()
missing_pct = 100 * missing / len(df)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

if missing.sum() == 0:
    print("✅ No missing values detected!")

# Duplicates
duplicates = df.duplicated().sum()
print(f"\nDUPLICATE ROWS: {duplicates}")
if duplicates == 0:
    print("✅ No duplicates detected!")

# Unique users
print(f"\nUNIQUE USERS: {df['user id'].nunique():,}")
print(f"Total Records: {len(df):,}")
if df['user id'].nunique() == len(df):
    print("✅ Each row represents a unique user!")


## 3. Conversion Rate Analysis

### Conversion Rate Formula

The conversion rate is calculated as:

$$CR = \frac{\text{Number of Conversions}}{\text{Total Users}}$$

### Relative Lift Formula

The relative lift measures the improvement of the test group over the control:

$$\text{Lift} = \frac{CR_{\text{test}} - CR_{\text{control}}}{CR_{\text{control}}} \times 100\%$$


In [ ]:
# Overall conversion
overall_cr = df['converted'].mean()
print(f"Overall Conversion Rate: {overall_cr:.4%}")

# By group
print("\nConversion Rate by Group:")
conversion_by_group = df.groupby('test group')['converted'].agg([
    ('Users', 'count'),
    ('Conversions', 'sum'),
    ('Conversion_Rate', 'mean')
])
conversion_by_group['Conversion_Rate'] = conversion_by_group['Conversion_Rate'] * 100
print(conversion_by_group)

# Calculate lift
cr_ad = df[df['test group'] == 'ad']['converted'].mean()
cr_psa = df[df['test group'] == 'psa']['converted'].mean()
lift = (cr_ad - cr_psa) / cr_psa * 100

print(f"\n📈 Conversion Lift (Ad vs PSA): {lift:+.2f}%")
print(f"   Ad Group:  {cr_ad:.6f} ({cr_ad*100:.2f}%)")
print(f"   PSA Group: {cr_psa:.6f} ({cr_psa*100:.2f}%)")


## 4. Dose-Response Analysis

We analyze the relationship between ad exposure and conversion rates. This helps identify:
- Optimal ad frequency
- Diminishing returns
- Correlation between exposure and conversion

### Correlation Analysis

We use both Pearson and Spearman correlation:

- **Pearson correlation**: Measures linear relationships
  $$r = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum (x_i - \bar{x})^2 \sum (y_i - \bar{y})^2}}$$

- **Spearman correlation**: Measures monotonic relationships (non-parametric)
  $$\rho = 1 - \frac{6 \sum d_i^2}{n(n^2 - 1)}$$
  
  where $d_i$ is the difference between ranks.


In [ ]:
# Visualization: Conversion Rate Comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
conversion_data = pd.DataFrame({
    'Group': ['Ad Group', 'PSA Group'],
    'Conversion Rate (%)': [cr_ad * 100, cr_psa * 100],
    'Users': [len(df[df['test group'] == 'ad']), len(df[df['test group'] == 'psa'])]
})

axes[0].bar(conversion_data['Group'], conversion_data['Conversion Rate (%)'], 
            color=['#3B82F6', '#6B7280'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Conversion Rate (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Conversion Rate by Group', fontsize=14, fontweight='bold', pad=20)
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
for i, (group, rate) in enumerate(zip(conversion_data['Group'], conversion_data['Conversion Rate (%)'])):
    axes[0].text(i, rate + 0.1, f'{rate:.4f}%', ha='center', fontweight='bold', fontsize=11)

# Lift visualization
axes[1].barh(['Relative Lift'], [lift], color='green' if lift > 0 else 'red', alpha=0.7, edgecolor='black', linewidth=1.5)
axes[1].axvline(x=0, color='black', linestyle='--', linewidth=1)
axes[1].set_xlabel('Lift (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Conversion Lift (Ad vs PSA)', fontsize=14, fontweight='bold', pad=20)
axes[1].text(lift, 0, f'{lift:+.2f}%', ha='left' if lift > 0 else 'right', va='center', 
             fontweight='bold', fontsize=12, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
axes[1].grid(axis='x', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()


In [ ]:
# Separate ad group for dose-response analysis
ad_group = df[df['test group'] == 'ad']

# Correlation between ads and conversion
correlation = ad_group['total ads'].corr(ad_group['converted'])
print(f"Pearson Correlation (Total Ads vs Conversion): {correlation:.4f}")

# Create ad exposure bins
ad_group['ad_bins'] = pd.cut(ad_group['total ads'], bins=10, labels=False)

# Conversion rate by ad exposure level
print("\nConversion Rate by Ad Exposure Level:")
dose_response = ad_group.groupby('ad_bins').agg({
    'total ads': ['min', 'max', 'mean', 'count'],
    'converted': ['sum', 'mean']
}).round(4)
dose_response.columns = ['Min_Ads', 'Max_Ads', 'Mean_Ads', 'Users', 'Conversions', 'Conversion_Rate']
dose_response['Conversion_Rate'] = dose_response['Conversion_Rate'] * 100
dose_response = dose_response.sort_values('Mean_Ads')
print(dose_response)

# Statistical tests
spearman_corr, spearman_p = spearmanr(ad_group['total ads'], ad_group['converted'])
pearson_corr, pearson_p = pearsonr(ad_group['total ads'], ad_group['converted'])

print(f"\n📈 Dose-Response Statistical Tests:")
print(f"   Spearman Correlation: {spearman_corr:.4f} (p-value: {spearman_p:.6f})")
print(f"   Pearson Correlation: {pearson_corr:.4f} (p-value: {pearson_p:.6f})")

if spearman_p < 0.05:
    print(f"   ✅ Significant dose-response relationship detected")
else:
    print(f"   ⚠️  No significant dose-response relationship")


In [ ]:
# Visualization: Temporal Patterns
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Day of week analysis
day_order = [0, 1, 2, 3, 4, 5, 6]
day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
day_cr = [day_conversion.loc[i, 'Conversion_Rate'] if i in day_conversion.index else 0 for i in day_order]

axes[0].bar(range(len(day_order)), day_cr, color='steelblue', alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].set_xticks(range(len(day_order)))
axes[0].set_xticklabels(day_labels)
axes[0].set_ylabel('Conversion Rate (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Conversion Rate by Day of Week', fontsize=14, fontweight='bold', pad=20)
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
for i, rate in enumerate(day_cr):
    if rate > 0:
        axes[0].text(i, rate + 0.05, f'{rate:.2f}%', ha='center', fontweight='bold', fontsize=10)

# Hour analysis (top 10)
top_hours = hour_conversion.sort_values('Conversion_Rate', ascending=False).head(10)
axes[1].barh(range(len(top_hours)), top_hours['Conversion_Rate'], 
             color='coral', alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1].set_yticks(range(len(top_hours)))
axes[1].set_yticklabels([f'Hour {h}' for h in top_hours.index])
axes[1].set_xlabel('Conversion Rate (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Top 10 Hours by Conversion Rate', fontsize=14, fontweight='bold', pad=20)
axes[1].grid(axis='x', alpha=0.3, linestyle='--')
for i, rate in enumerate(top_hours['Conversion_Rate']):
    axes[1].text(rate + 0.05, i, f'{rate:.2f}%', va='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()


## 5. Temporal Patterns Analysis

We analyze conversion rates by:
- Day of week
- Hour of day
- Day-hour combinations

This helps identify optimal timing for ad delivery.


## Summary & Key Findings

### Data Quality
- ✅ Dataset is clean with no missing values or duplicates
- ✅ Each row represents a unique user
- ✅ Groups are properly randomized

### Conversion Insights
- Overall conversion rate: {overall_cr:.4%}
- Ad group conversion rate: {cr_ad:.4%}
- PSA group conversion rate: {cr_psa:.4%}
- Observed lift: {lift:+.2f}%

### Next Steps
1. Statistical significance testing (see `ab_test_frequentist.ipynb`)
2. Bayesian analysis for probability estimates (see `ab_test_bayesian.ipynb`)
3. Business impact assessment (see `ab_test_business_impact.ipynb`)


In [ ]:
# Conversion rates by day of week
print("Conversion Rates by Day of Week:")
day_conversion = df.groupby('most ads day').agg({
    'converted': ['count', 'sum', 'mean']
}).round(4)
day_conversion.columns = ['Users', 'Conversions', 'Conversion_Rate']
day_conversion['Conversion_Rate'] = day_conversion['Conversion_Rate'] * 100
print(day_conversion.sort_index())

# Conversion rates by hour
print("\nConversion Rates by Hour (Top 10):")
hour_conversion = df.groupby('most ads hour').agg({
    'converted': ['count', 'sum', 'mean']
}).round(4)
hour_conversion.columns = ['Users', 'Conversions', 'Conversion_Rate']
hour_conversion['Conversion_Rate'] = hour_conversion['Conversion_Rate'] * 100
print(hour_conversion.sort_values('Conversion_Rate', ascending=False).head(10))

# Day and hour combination analysis
print("\nConversion Rates by Day and Hour (Top 10):")
day_hour_conversion = df.groupby(['most ads day', 'most ads hour']).agg({
    'converted': ['count', 'sum', 'mean']
}).round(4)
day_hour_conversion.columns = ['Users', 'Conversions', 'Conversion_Rate']
day_hour_conversion['Conversion_Rate'] = day_hour_conversion['Conversion_Rate'] * 100
day_hour_conversion = day_hour_conversion.sort_values('Conversion_Rate', ascending=False)
print(day_hour_conversion.head(10))
